In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from tqdm import tqdm

In [12]:
response = requests.get('https://lifehacker.ru/analog-discord/')
response.text

'<!doctype html>\n<html data-n-head-ssr lang="ru-RU" data-n-head="%7B%22lang%22:%7B%22ssr%22:%22ru-RU%22%7D%7D">\n  <head >\n    <title>7 крутых аналогов Discord: подбираем замену — Лайфхакер</title><meta data-n-head="ssr" charset="utf-8"><meta data-n-head="ssr" name="viewport" content="width=device-width, initial-scale=1"><meta data-n-head="ssr" name="format-detection" content="telephone=no,date=no,address=no,email=no,url=no"><meta data-n-head="ssr" data-hid="al:ios:app_name" name="al:ios:app_name" content="Лайфхакер"><meta data-n-head="ssr" name="apple-itunes-app" content="app-id=943373933"><meta data-n-head="ssr" data-hid="al:ios:app_store_id" name="al:ios:app_store_id" content="943373933"><meta data-n-head="ssr" data-hid="og:publisher" property="og:publisher" content="//www.facebook.com/lifehacker.ru"><meta data-n-head="ssr" data-hid="og:site_name" property="og:site_name" content="Лайфхакер"><meta data-n-head="ssr" data-hid="og:type" property="og:type" content="website"><meta data-

In [25]:
base_url = 'https://lifehacker.ru/topics/technology/' # базовая часть ссылки, позже в цикле будем добавлять к ней пагинацию
response = requests.get('https://lifehacker.ru/topics/technology/?page=1') # получаем контент первой страиниц
soup = BeautifulSoup(response.text, 'lxml') # инициализируем объект bs4 и задаем парсер lxml

In [29]:
raw_items = soup.find_all('a', class_='lh-article-card-controls__comments')  # ищем все a-элементы с классом lh-article-card-controls__comments
links = [item.get('href').replace('#comments', '') for item in raw_items] # получаем у ссылок только href-атрибут, убираем подстроку '/#comments'
links

['/analog-discord/',
 '/bolshoe-obnovlenie-onedrive/',
 '/luchshie-nedorogie-smartfony-2024/',
 '/ipod-classic/',
 '/discord-oficialno-zablokirovali-v-rossii/',
 '/nobelevskaya-premiya-po-fizike-2024/',
 '/vkladki-v-google-docs/',
 '/otnosheniya-vladelcev-iphone-i-android/',
 '/anons-ikoffy-gobrew/',
 '/anons-samsung-galaxy-a16-5g/',
 '/krupneishee-obnovlenie-vk-muzyki/',
 '/google-pixel-9-raskryvaet-mestopolozhenie/',
 '/prodvinutaya-zashhita-android-ot-krazhi/',
 '/rasshireniya-chrome-vybor-redakcii/',
 '/kakuyu-kameru-kupit-dlya-videonablyudeniya-doma/',
 '/podarki-v-telegram/',
 '/arc-search-na-android/',
 '/ps5-vs-ps5-pro-za-i-protiv/',
 '/analogi-chatgpt/',
 '/chatgpt-canvas/',
 '/obzor-huawei-watch-d2-chasov-s-izmereniem-davleniya/',
 '/poisk-po-video-v-google-obektive/',
 '/anons-anker-soundcore-liberty-4-pro/',
 '/prilozheniya-android-sentyabr-2024/',
 '/gostevoi-rezhim-v-iphone/',
 '/kak-vybrat-saundbar/',
 '/skladnoi-smartfon-microsoft/',
 '/dumayushhii-ii-google/',
 '/neiro

In [34]:
parsed_urls = []
for page_num in range(1, 11):
    url = f'{base_url}/?page={page_num}' # подставляем нужный номер страницы для пагинации, дальше парсим каждую страницу аналогично коду выше
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'lxml')
    raw_items = soup.find_all('a', class_='lh-article-card-controls__comments')
    parsed_urls += [item.get('href').replace('#comments', '') for item in raw_items]
len(parsed_urls) # 10 страниц X 20 статей

297

In [37]:
parsed_urls[159:168]

['/anons-huawei-mate-xt/',
 '/wix-uxodit-iz-rossii/',
 '/ceny-iphone-16-v-rossii/',
 '/data-vyxoda-ios-18/',
 '/apple-predstavila-airpods-4-i-airpods-max-vtorogo-pokoleniya/',
 '/prezentaciya-iphone-16/',
 '/apple-watch-series-10-anons/',
 '/krea-ai/',
 '/anons-capsule-solo/']

In [43]:
#Спарсим заголовок и текст статьи из полученных ссылок

base_url = 'https://lifehacker.ru'
response = requests.get('https://lifehacker.ru/analog-discord/')
soup = BeautifulSoup(response.text, 'lxml')
soup.find('div', class_='single-article__post-content single-article__content-container').text

' Оглавление  \n            Revolt\n          \n            Guilded\n          \n            Matrix\n          \n            Quebick\n          \n            Valour\n          \n            TeamSpeak\n          \n            Mumble\n            Оглавление   \n            Revolt\n          \n            Guilded\n          \n            Matrix\n          \n            Quebick\n          \n            Valour\n          \n            TeamSpeak\n          \n            Mumble\n           Как и предполагалось, Роскомнадзор полностью заблокировал Discord. Сервис полюбился многим своей простотой и одновременно богатыми возможностями. Он позволял общаться в виртуальных комнатах с помощью текста, голосовых и видеовызовов, а также стримить игры и демонстрировать экран. В Discord также удобно было создавать сообщества по интересам с функцией назначения ролей, реакциями и опросами. Всё это работало бесплатно как на компьютере, так и на мобильных устройствах.Полного аналога платформы, к сожалению, н

In [57]:
soup.find('h1', class_='article-card__title').text

'7 крутых аналогов Discord: подбираем замену'

In [59]:
result = []

for url in tqdm(parsed_urls):
    article = {}

    article_url = f'{base_url}{url}' # подставляем часть ссылки, ведущую материал
    response = requests.get(article_url)
    soup = BeautifulSoup(response.text, 'lxml')

    article['title'] = soup.find('h1', class_='article-card__title').text # Получаем текст из span в первом h1 с классом tm-title_h1
    article['text'] = soup.find('div', class_='single-article__post-content single-article__content-container').text # Получаем весь текст из div с классом tm-article-body

    result.append(article)

100%|██████████| 297/297 [07:26<00:00,  1.50s/it]


In [60]:
#датафрейм

pd.set_option('display.max_colwidth', 400)
data = pd.DataFrame(result)
data.head()

,title,text
0,7 крутых аналогов Discord: подбираем замену,Оглавление \n Revolt\n \n Guilded\n \n Matrix\n \n Quebick\n \n Valour\n \n TeamSpeak\n \n Mumble\n Оглавление \n Revolt\n \n Guilded\n \n Matrix\n \n Quebick\n \n ...
1,Новое мобильное приложение и улучшенный поиск: для OneDrive вышло большое обновление,"Microsoft анонсировала большое обновление для облачного хранилища OneDrive. В ближайшие недели пользователям сервиса станут доступны улучшенный поиск, новое мобильное приложение и множество других нововведений.По словам компании, одна из главных новинок — улучшенный поиск. Благодаря новым фильтрам и более подробным результатам найти определённый файл или папку станет гораздо проще. Это касаетс..."
2,10 крутых смартфонов до 30 000 рублей в 2024 году,Оглавление \n Оптимальный баланс цены и качества\n \n Хорошая камера\n \n Живучий аккумулятор\n \n Максимальная производительность\n \n Эффектный дизайн\n Оглавление \n Оптимальный баланс цены и качества\n \n Хорошая камера\n \n Живучий ак...
3,Почему я всё ещё пользуюсь старым добрым iPod Classic: 7 причин,Оглавление \n Ностальгия и эстетика\n \n Тот самый звук\n \n Простота использования\n \n Автономность\n \n Независимость от интернета и спокойствие\n \n Надёжность и долговечность\n \n Возможность модификации\n Оглавление \n Ностальгия и э...
4,Discord официально заблокировали в России,"Роскомнадзор сообщил о блокировке сервиса Discord на территории России. Решение связано с нарушениями требований российского законодательства, пишет ТАСС.Ранее в РКН отмечали, что требовали от сервиса удаления 947 страниц с различными противоправными материалами. Discord не выполнил все требования ведомства, сохранив часть контента.Также в 2023 году за неудаление противоправной информации суд ..."
